In [1]:
from datasets import coco_parse
from datasets import flickr8k_parse
from keras import Model 
from keras.applications import VGG16
from tensorflow.python.client import device_lib

import image_preprocessing
import numpy as np
import os
import time
import transfer_models

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Image encoding

Currently, an image encoder is built using VGG16 architecture pre-trained on imagenet database.

The features were obtained from the "fc2" layer - last fully-connected layer before the predictions layer.

Generated features in numpy arrays for both training and validation datasets were saved to .npy files.

### COCO dataset

In [2]:
# captions_path = 'D:/coco/annotations/'
# images_path = 'D:/coco/images/'

# # parse JSON file with captions to get paths to images with captions

# val_filenames_with_captions = coco_parse.get_image_filename_with_caption(captions_path, images_path, 
#                                                                      train=False)

# val_filenames_with_all_captions = coco_parse.get_image_with_all_captions(val_filenames_with_captions)

# train_filenames_with_captions = coco_parse.get_image_filename_with_caption(captions_path, images_path, 
#                                                                      train=True)
# train_filenames_with_all_captions = coco_parse.get_image_with_all_captions(train_filenames_with_captions)

### Flickr8k dataset

In [3]:
images_path = 'D:/Flickr8k/images/'
annotations_path = 'D:/Flickr8k/annotations/'
captions_file = 'D:/Flickr8k/annotations/Flickr8k.token.txt'
train_txt_path = 'D:/Flickr8k/annotations/Flickr_8k.trainImages.txt'
dev_txt_path = 'D:/Flickr8k/annotations/Flickr_8k.devImages.txt'
test_txt_path = 'D:/Flickr8k/annotations/Flickr_8k.testImages.txt'

filenames_with_all_captions = flickr8k_parse.generate_filenames_with_all_captions(captions_file, images_path)

train_filenames_with_all_captions = flickr8k_parse.generate_set(train_txt_path, filenames_with_all_captions, images_path)
val_filenames_with_all_captions = flickr8k_parse.generate_set(dev_txt_path, filenames_with_all_captions, images_path)
test_filenames_with_all_captions = flickr8k_parse.generate_set(test_txt_path, filenames_with_all_captions, images_path)

In [4]:
print('Number of images in validation dataset: {}'.format(len(val_filenames_with_all_captions)))
print('Number of images in training dataset: {}'.format(len(train_filenames_with_all_captions)))

Number of images in validation dataset: 1000
Number of images in training dataset: 6000


In [5]:
attn = True

In [6]:
### encode features for validation images
start = time.time()
val_transfer_values = transfer_models.use_pretrained_model_for_images(val_filenames_with_all_captions, attn, batch_size=16)
time_val = time.time() - start

Instructions for updating:
Colocations handled automatically by placer.
5% of images processed
10% of images processed
15% of images processed
20% of images processed
25% of images processed
30% of images processed
35% of images processed
40% of images processed
45% of images processed
50% of images processed
55% of images processed
60% of images processed
65% of images processed
70% of images processed
75% of images processed
80% of images processed
85% of images processed
90% of images processed
95% of images processed
100% of images processed
105% of images processed


In [7]:
print('Validation dataset encoding took {:.1f} minutes'.format(time_val / 60))

Validation dataset encoding took 1.4 minutes


In [8]:
### save features for validation images
transfer_models.save_features(val_transfer_values, './cnn_features/', 'vgg16_flickr8k_val_attn.npy')

Array was saved to ./cnn_features/vgg16_flickr8k_val_attn.npy


In [9]:
a = np.load('./cnn_features/vgg16_flickr8k_val_attn.npy')
(a == val_transfer_values).all()

True

In [10]:
### encode features for training images
start = time.time()
train_transfer_values = transfer_models.use_pretrained_model_for_images(train_filenames_with_all_captions, attn, batch_size=16)
time_train = time.time() - start

Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000001F8AEBDE748>>
Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "D:\Anaconda\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.


5% of images processed
10% of images processed
15% of images processed
20% of images processed
25% of images processed
30% of images processed
35% of images processed
40% of images processed
45% of images processed
50% of images processed
55% of images processed
60% of images processed
65% of images processed
70% of images processed
75% of images processed
80% of images processed
85% of images processed
90% of images processed
95% of images processed
100% of images processed


In [11]:
print('Training dataset encoding took {:.1f} minutes'.format(time_train / 60))

Training dataset encoding took 9.3 minutes


In [12]:
transfer_models.save_features(train_transfer_values, './cnn_features/', 'vgg16_flickr8k_train_attn.npy')

Array was saved to ./cnn_features/vgg16_flickr8k_train_attn.npy


In [13]:
a = np.load('./cnn_features/vgg16_flickr8k_train_attn.npy')
(a == train_transfer_values).all()

True

In [14]:
### encode features for training images
start = time.time()
test_transfer_values = transfer_models.use_pretrained_model_for_images(test_filenames_with_all_captions, attn, batch_size=16)
time_train = time.time() - start

Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x000001F8AF422198>>
Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "D:\Anaconda\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.


5% of images processed
10% of images processed
15% of images processed
20% of images processed
25% of images processed
30% of images processed
35% of images processed
40% of images processed
45% of images processed
50% of images processed
55% of images processed
60% of images processed
65% of images processed
70% of images processed
75% of images processed
80% of images processed
85% of images processed
90% of images processed
95% of images processed
100% of images processed
105% of images processed


In [15]:
transfer_models.save_features(test_transfer_values, './cnn_features/', 'vgg16_flickr8k_test_attn.npy')

Array was saved to ./cnn_features/vgg16_flickr8k_test_attn.npy
